# 概要
以下Notebookを元に日本語にさせていただいています。ありがとうございます。  
また、自分用に変数名を変更したり、メモしたりしていますのでご注意ください。
![](http://)https://www.kaggle.com/robikscube/m5-forecasting-starter-data-exploration

# M5 Forecasting Challenge
<img src="https://images.ctfassets.net/osv85d77hkdf/7LsZ5bZzvGaG6iwYkoKEUc/84afe0bf84371542fe56e6d5f0b3377b/hero_telescope_01_2x.png" width="500" height="300" />

このnotebookのゴールは2020年のM5コンペをすばやく理解してもらうことだ。
読み終えれば、あなたは問題を解く良い目標を思いつき、提供されたデータとその評価を得られるだろう。

注意:
- 2つの平行して実施されるコンペがある。： **Accuracy** と **Uncertainty**
    - **Accuracy** は評価指標にWeightedRMSSE（Weighted Root Mean Squared Scaled Error）
    - **Uncertainty** は評価指標にWeightedWPL（Weighted Scaled Pinball Loss）
- Wal-Martの階層的販売データの予測が仕事である
- そのデータは3つの州（カリフォルニア州、テキサス州、ウィスコンシン州）のデータと商品レベル（item level）、部門（department）、商品カテゴリー（product catgories）、店の詳細が入っている
- 加えて、説明変数である価格、プロモーション、曜日、スペシャルイベントなどがある

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from itertools import cycle

# 参考：グラフ作成のためのチートシートとPythonによる各種グラフの実装
# https://qiita.com/4m1t0/items/76b0033edb545a78cef5
# 最大列
pd.set_option('max_columns', 50)
# https://matplotlib.org/3.1.3/gallery/style_sheets/bmh.html
# 「Bayesian Methods for Hackers style sheet」
plt.style.use('bmh')
# 「rcParams」はmatplotlibのデフォルトセット
# 「axes.prop_cycle」はcycler.Cycler型
color_pal = plt.rcParams['axes.prop_cycle'].by_key()['color']
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

# Data Files
- `calendar.csv` - 各日の情報.
- `sales_train_validation.csv` - 各日および店での販売個数[d_1 - d_1913]
- `sample_submission.csv` - 現在の提出フォーマット。「Evaluation」タブで詳細を見れる。
- `sell_prices.csv` - 店、日付単位での販売価格

Not available yet:
- `sales_train_evaluation.csv` - コンペの提出期限の1ヶ月前にて字される。[d_1 - d_1941]

In [ ]:
!ls -GFlash --color ../input/m5-forecasting-accuracy/

In [ ]:
# データ読み込み
INPUT_DIR = '../input/m5-forecasting-accuracy'
# cal = pd.read_csv(f'{INPUT_DIR}/calendar.csv')
# stv = pd.read_csv(f'{INPUT_DIR}/sales_train_validation.csv')
# ss = pd.read_csv(f'{INPUT_DIR}/sample_submission.csv')
# sellp = pd.read_csv(f'{INPUT_DIR}/sell_prices.csv')

df_calendar = pd.read_csv(f'{INPUT_DIR}/calendar.csv')
df_sales_train_validation = pd.read_csv(f'{INPUT_DIR}/sales_train_validation.csv')
df_sample_submission = pd.read_csv(f'{INPUT_DIR}/sample_submission.csv')
df_sell_prices = pd.read_csv(f'{INPUT_DIR}/sell_prices.csv')

In [ ]:
df_calendar.head()

In [ ]:
df_sales_train_validation.head()

In [ ]:
df_sample_submission.head()

In [ ]:
df_sell_prices.head()

# 何を正確に予測するのか?
28日間の販売個数を予測しようとしている。サンプル提出ファイルは以下のフォーマットになっている:
- 列は28日分ある。私達の予測でこれらを埋める。
- 各行は特定の商品を表す。このIDは商品タイプ、州、店を示す。が、正確には商品を知らない。

In [ ]:
df_sample_submission.head()

sales_train_validation.csvのデータセットには販売履歴データが与えられている。
- 行はd_1〜d_1913日間のデータセットが存在する。商品の部門、カテゴリー、州、店のIDがある。
- d_1914 - d_1941はステージ1で予測する**validation**の行です。
- d_1942 - d_1969は最後に予測する**evaluation**の行です。

In [ ]:
df_sales_train_validation.head()

# 1商品のデータを見える化する
- たくさん販売している商品をランダムに鳥、学習データを通してどのように売られているか見ましょう
- `FOODS_3_090_CA_3_validation`がたくさん売られていますね。
- 売れずに低迷している日があることに注意しましょう。

In [ ]:
# 「日付」の列を取り出す
d_cols = [column for column in df_sales_train_validation.columns if 'd_' in column]

In [ ]:
# ↓にpandasで以下を実行する
# 1. 商品を選択
# 2. IDにindexを設定し、売上データのみの列にする
#   https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.set_index.html
# 3. 転置する
# 4. データをプロットする

df_sales_train_validation.loc[
    df_sales_train_validation['id'] == 'FOODS_3_090_CA_3_validation'
] \
.set_index('id')[d_cols] \
.T \
.plot(figsize=(15, 5),
      title='FOODS_3_090_CA_3 sales by "d" number',
      color=next(color_cycle))
plt.legend('')
plt.show()

# 見た感じは250日目ぐらいまでほとんど売れてないね...

## 実際の日とデータをマージする
- 過去と未来の日付の追加情報を持つカレンダーが与えられている
- カレンダーデータは各日のデータとマージできる
- ここから週および年の流行がわかるだろう

In [ ]:
# カレンダーではこのようになっている
# 列は今注目している列のみ
df_calendar[
    [
    # 日付ID（d_1〜）
    'd',
    # 実際の日付
    'date',
    # イベント名1
    'event_name_1',
    # イベント名2
    'event_name_2',
    # イベントタイプ1
    'event_type_1',
    # イベントタイプ2
    'event_type_2', 
    # カリフォルニア州のSNAP
    'snap_CA',
    ]
].head()

In [ ]:
# 商品データとカレンダーをマージする
# 先程のデータ取り出し
df_example = df_sales_train_validation.loc[
    df_sales_train_validation['id'] == 'FOODS_3_090_CA_3_validation'] \
[d_cols].T
# 列名（元のindex名）を正しくする
df_example = df_example.rename(columns={8412:'FOODS_3_090_CA_3'})
# インデックス値をリセット
# 名称を「d」とする
df_example = df_example.reset_index().rename(columns={'index': 'd'}) # make the index "d"
# カレンダー情報のマージ
# 「validate」は1:1のマージであることをチェックする
df_example = df_example.merge(df_calendar, how='left', validate='1:1')
# indexにdate（calendar.csvの日付）を設定する
df_example.set_index('date')['FOODS_3_090_CA_3'] \
    .plot(figsize=(15, 5),
          color=next(color_cycle),
          title='FOODS_3_090_CA_3 sales by actual sale dates')
plt.show()

# 2011-10-05あたりから売上てるのがわかる

In [ ]:
# 他のトップセールスの例を見てみよう！(1)
# HOBBIES_1_234_CA_3_validation
df_example2 = df_sales_train_validation.loc[
    df_sales_train_validation['id'] == 'HOBBIES_1_234_CA_3_validation'
][d_cols].T
df_example2 = df_example2.rename(columns={6324:'HOBBIES_1_234_CA_3'}) # Name it correctly
df_example2 = df_example2.reset_index().rename(columns={'index': 'd'}) # make the index "d"
df_example2 = df_example2.merge(df_calendar, how='left', validate='1:1')
df_example2.set_index('date')['HOBBIES_1_234_CA_3'] \
    .plot(figsize=(15, 5),
          color=next(color_cycle),
          title='HOBBIES_1_234_CA_3 sales by actual sale dates')
plt.show()

In [ ]:
# 他のトップセールスの例を見てみよう！(2)
# HOUSEHOLD_1_118_CA_3_validation
df_example3 = df_sales_train_validation.loc[
    df_sales_train_validation['id'] == 'HOUSEHOLD_1_118_CA_3_validation'
][d_cols].T
df_example3 = df_example3.rename(columns={6776:'HOUSEHOLD_1_118_CA_3'}) # Name it correctly
df_example3 = df_example3.reset_index().rename(columns={'index': 'd'}) # make the index "d"
df_example3 = df_example3.merge(df_calendar, how='left', validate='1:1')
df_example3.set_index('date')['HOUSEHOLD_1_118_CA_3'] \
    .plot(figsize=(15, 5),
          color=next(color_cycle),
          title='HOUSEHOLD_1_118_CA_3 sales by actual sale dates')
plt.show()


# 時間変数で売上を分解する
- 今、例で挙げた商品が以下単位でどのように売られているか見てみよう。
    - 1週間
    - 1月
    - 1年

In [ ]:
# サンプル商品IDリスト
example_ids = ['FOODS_3_090_CA_3','HOBBIES_1_234_CA_3','HOUSEHOLD_1_118_CA_3']
# サンプル商品の売上データ
df_example_all = [
    df_example, 
    df_example2, 
    df_example3
]

# 各商品データをプロットしていく
for i in [0, 1, 2]:
    # グラフ準備
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 3))
    
    # 週単位で平均を取る
    df_example_all[i].groupby('wday').mean()[
        example_ids[i]] \
        .plot(
            kind='line',
            title='average sale: day of week',
            lw=5,
            color=color_pal[0],
            ax=ax1
    )
    
    # 月単位で平均を取る
    df_example_all[i].groupby('month').mean()[
        example_ids[i]] \
        .plot(kind='line',
              title='average sale: month',
              lw=5,
              color=color_pal[4],
              ax=ax2)
    
    # 年で平均を取る
    df_example_all[i].groupby('year').mean()[example_ids[i]] \
        .plot(kind='line',
              lw=5,
              title='average sale: year',
              color=color_pal[2],
              ax=ax3)
    
    # サブタイトル
    fig.suptitle(f'Trends for item: {example_ids[i]}',
                 size=20,
                 y=1.1)
    plt.tight_layout()
    plt.show()

# 他の商品も見てみよう！
- 一緒に異なる20商品を出してみよう
- それらのグラフからいくつかの考察が得られるだろう
    - 共通して言えるのは一定期間、購入されていない商品がある。
    - 1日に売れる量が1つ、または少ない商品があり、それは予測が難しいだろう
    - 他の商品は急な需要が見られる（スーパーボウルの日曜か？）、もしかしたら与えられている"イベント"のデータが役立つかもしれない。

In [ ]:
# 20例のデータ取り出し
twenty_examples = df_sales_train_validation.sample(
        20, random_state=529
    ) \
    .set_index('id')[d_cols] \
    .T \
    .merge(df_calendar.set_index('d')['date'],
           left_index=True,
           right_index=True,
            validate='1:1') \
    .set_index('date')

In [ ]:
# 20例のデータをプロット
fig, axs = plt.subplots(10, 2, figsize=(15, 20))
axs = axs.flatten()
ax_idx = 0
for item in twenty_examples.columns:
    twenty_examples[item].plot(title=item,
                              color=next(color_cycle),
                              ax=axs[ax_idx])
    ax_idx += 1
plt.tight_layout()
plt.show()

# 商品タイプ別の売上高の推移
- いくつかの商品タイプがある
    - Hobbies（娯楽品）
    - Household（日用品）
    - Foods（食べ物）
- 各商品タイプごとの需要をプロットしてみよう

In [ ]:
# 何の商品タイプがあるか
df_sales_train_validation['cat_id'].unique()

In [ ]:
# カテゴリ別でどれだけ商品数があるか
df_sales_train_validation.groupby('cat_id').count()['id'] \
    .sort_values() \
    .plot(kind='barh', figsize=(15, 5), title='Count of Items by Category')
plt.show()

In [ ]:
# 過去の売上
# index: 日付ID
# column: 商品ID
past_sales = df_sales_train_validation.set_index('id')[d_cols] \
    .T \
    .merge(df_calendar.set_index('d')['date'],
           left_index=True,
           right_index=True,
            validate='1:1') \
    .set_index('date')


# カテゴリ別で各商品データを取り出して
for i in df_sales_train_validation['cat_id'].unique():
    # 対象カテゴリの商品列を取り出す
    items_col = [c for c in past_sales.columns if i in c]
    # 対象カテゴリの商品数を合計してプロット
    past_sales[items_col] \
        .sum(axis=1) \
        .plot(figsize=(15, 5),
              alpha=0.8,
              title='Total Sales by Item Type')
plt.legend(df_sales_train_validation['cat_id'].unique())
plt.show()

# 店ごとの売上
10店舗のデータが提供されている。店舗ごとの売上数はいくらか？
- いくつかの店舗は他店舗よりも一定の売上がある
- 「CA_2」の店舗は2015年に大きな変化が起こったようだ

In [ ]:
# 店リスト
store_list = df_sell_prices['store_id'].unique()

# 店単位で集計
for s in store_list:
    # 店単位で商品取り出し
    # ※ IDに店のIDが含まれている
    store_items = [c for c in past_sales.columns if s in c]
    # 店単位で移動平均を出す
    # https://note.nkmk.me/python-pandas-rolling/
    # 店で売っている商品を日付ごとに合計して90日ずつの移動平均を表示
    past_sales[store_items] \
        .sum(axis=1) \
        .rolling(90).mean() \
        .plot(figsize=(15, 5),
              alpha=0.8,
              title='Rolling 90 Day Average Total Sales (10 stores)')
plt.legend(store_list)
plt.show()

- 同じデータを異なる側面で見るため、店舗ごとの7日間の需要を移動平均でプロットする
- 明らかにいくつかの店舗は急激な変化をしていて、店の拡張をしたり、競合店が近くにできたりしている可能性がある
- どちらにせよ、需要パターンは予測するモデルを作る時には重要である

In [ ]:
# 店で売っている商品を日付ごとに合計して7日ずつの移動平均を表示
fig, axes = plt.subplots(5, 2, figsize=(15, 10), sharex=True)
axes = axes.flatten()
ax_idx = 0
for s in store_list:
    store_items = [c for c in past_sales.columns if s in c]
    past_sales[store_items] \
        .sum(axis=1) \
        .rolling(7).mean() \
        .plot(alpha=1,
              ax=axes[ax_idx],
              title=s,
              lw=3,
              color=next(color_cycle))
    ax_idx += 1
# plt.legend(store_list)
plt.suptitle('Weekly Sale Trends by Store ID')
plt.tight_layout()
plt.show()

# 売上とカレンダーのヒートマップ

In [ ]:
# ----------------------------------------------------------------------------
# Author:  Nicolas P. Rougier
# License: BSD
# ----------------------------------------------------------------------------
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from datetime import datetime
from dateutil.relativedelta import relativedelta


def calmap(ax, year, data):
    ax.tick_params('x', length=0, labelsize="medium", which='major')
    ax.tick_params('y', length=0, labelsize="x-small", which='major')

    # Month borders
    xticks, labels = [], []
    start = datetime(year,1,1).weekday()
    for month in range(1,13):
        first = datetime(year, month, 1)
        last = first + relativedelta(months=1, days=-1)

        y0 = first.weekday()
        y1 = last.weekday()
        x0 = (int(first.strftime("%j"))+start-1)//7
        x1 = (int(last.strftime("%j"))+start-1)//7

        P = [ (x0,   y0), (x0,    7),  (x1,   7),
              (x1,   y1+1), (x1+1,  y1+1), (x1+1, 0),
              (x0+1,  0), (x0+1,  y0) ]
        xticks.append(x0 +(x1-x0+1)/2)
        labels.append(first.strftime("%b"))
        poly = Polygon(P, edgecolor="black", facecolor="None",
                       linewidth=1, zorder=20, clip_on=False)
        ax.add_artist(poly)
    
    ax.set_xticks(xticks)
    ax.set_xticklabels(labels)
    ax.set_yticks(0.5 + np.arange(7))
    ax.set_yticklabels(["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"])
    ax.set_title("{}".format(year), weight="semibold")
    
    # Clearing first and last day from the data
    valid = datetime(year, 1, 1).weekday()
    data[:valid,0] = np.nan
    valid = datetime(year, 12, 31).weekday()
    # data[:,x1+1:] = np.nan
    data[valid+1:,x1] = np.nan

    # Showing data
    ax.imshow(data, extent=[0,53,0,7], zorder=10, vmin=-1, vmax=1,
              cmap="RdYlBu_r", origin="lower", alpha=.75)

- クリスマスにはウォルマートは閉まるようだ
- 最も需要が高い日は2016年3月6日の日曜だ
- 何があったんだろうと思うかもしれないが...もしかしたら、CNN主催の7回目の民主党党首の候補のディベート（https://www.onthisday.com/date/2016/march/6）
    - んなわけない :D

In [ ]:
# 最低売上
print('The lowest sale date was:', past_sales.sum(axis=1).sort_values().index[0],
     'with', past_sales.sum(axis=1).sort_values().values[0], 'sales')
# 最高売上 ※ 元はlowestだけどhighestに変更
print('The highest sale date was:', past_sales.sum(axis=1).sort_values(ascending=False).index[0],
     'with', past_sales.sum(axis=1).sort_values(ascending=False).values[0], 'sales')

In [ ]:
# ヒートマップ作成
from sklearn.preprocessing import StandardScaler
sscale = StandardScaler()
past_sales.index = pd.to_datetime(past_sales.index)

for i in df_sales_train_validation['cat_id'].unique():
    # 2013年
    fig, axes = plt.subplots(3, 1, figsize=(20, 8))
    items_col = [c for c in past_sales.columns if i in c]
    sales2013 = past_sales.loc[past_sales.index.isin(pd.date_range('31-Dec-2012',
                                                                   periods=371))][items_col].mean(axis=1)
    vals = np.hstack(sscale.fit_transform(sales2013.values.reshape(-1, 1)))
    calmap(axes[0], 2013, vals.reshape(53,7).T)
    
    # 2014年
    sales2014 = past_sales.loc[past_sales.index.isin(pd.date_range('30-Dec-2013',
                                                                   periods=371))][items_col].mean(axis=1)
    vals = np.hstack(sscale.fit_transform(sales2014.values.reshape(-1, 1)))
    calmap(axes[1], 2014, vals.reshape(53,7).T)
    
    # 2015年
    sales2015 = past_sales.loc[past_sales.index.isin(pd.date_range('29-Dec-2014',
                                                                   periods=371))][items_col].mean(axis=1)
    vals = np.hstack(sscale.fit_transform(sales2015.values.reshape(-1, 1)))
    calmap(axes[2], 2015, vals.reshape(53,7).T)
    
    plt.suptitle(i, fontsize=30, x=0.4, y=1.01)
    plt.tight_layout()
    plt.show()

ヒートマップから面白いことがわかる:
- 食品は月末になるほど購入量が少なくなる。月の初めに給料の支払いがあるから？
- 日用品と娯楽品は1月に少なくなる。これは連休シーズンの後だから。
- 明らかに週末は買い物日だ（商品カテゴリーに無関係）

# 販売価格
各商品の階層的販売価格が与えられている。以前から取り上げている商品を見てみよう
- この商品の価格は上昇している
- 他店舗では異なる価格で販売されている

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
stores = []

# 1商品の店舗・週別の価格をプロット
for store, d in df_sell_prices.query('item_id == "FOODS_3_090"').groupby('store_id'):
    d.plot(x='wm_yr_wk',
          y='sell_price',
          style='.',
          color=next(color_cycle),
          figsize=(15, 5),
          title='FOODS_3_090 sale price over time',
         ax=ax,
          legend=store)
    stores.append(store)
    plt.legend()
plt.legend(stores)
plt.show()

In [ ]:
df_sell_prices['Category'] = df_sell_prices['item_id'].str.split('_', expand=True)[0]
fig, axs = plt.subplots(1, 3, figsize=(15, 4))
i = 0

# 店舗・カテゴリ別の価格をプロット
for cat, d in df_sell_prices.groupby('Category'):
    ax = d['sell_price'].apply(np.log1p) \
        .plot(kind='hist',
                         bins=20,
                         title=f'Distribution of {cat} prices',
                         ax=axs[i],
                                         color=next(color_cycle))
    ax.set_xlabel('Log(price)')
    i += 1
plt.tight_layout()

# 簡単な提出
- 過去30日間の平均を提出してみる

In [ ]:
# 過去30日間の平均を取った辞書データ
thirty_day_avg_map = df_sales_train_validation.set_index('id')[d_cols[-30:]].mean(axis=1).to_dict()

# 予測日の列リスト
fcols = [f for f in df_sample_submission.columns if 'F' in f]

for f in fcols:
    # 各商品単位で平均の値を設定
    df_sample_submission[f] = df_sample_submission['id'].map(thirty_day_avg_map).fillna(0)
    
df_sample_submission.to_csv('submission.csv', index=False)

#### submission.csvの例
id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
HOBBIES_1_001_CA_1_validation,0.9666666666666667,0.9666666666666667,0.9666666666666667,0.9666666666666667,0.9666666666666667,0.9666666666666667,0.9666666666666667,0.9666666666666667,0.9666666666666667,0.9666666666666667,0.9666666666666667,0.9666666666666667,0.9666666666666667,0.9666666666666667,0.9666666666666667,0.9666666666666667,0.9666666666666667,0.9666666666666667,0.9666666666666667,0.9666666666666667,0.9666666666666667,0.9666666666666667,0.9666666666666667,0.9666666666666667,0.9666666666666667,0.9666666666666667,0.9666666666666667,0.9666666666666667
HOBBIES_1_002_CA_1_validation,0.13333333333333333,0.13333333333333333,0.13333333333333333,0.13333333333333333,0.13333333333333333,0.13333333333333333,0.13333333333333333,0.13333333333333333,0.13333333333333333,0.13333333333333333,0.13333333333333333,0.13333333333333333,0.13333333333333333,0.13333333333333333,0.13333333333333333,0.13333333333333333,0.13333333333333333,0.13333333333333333,0.13333333333333333,0.13333333333333333,0.13333333333333333,0.13333333333333333,0.13333333333333333,0.13333333333333333,0.13333333333333333,0.13333333333333333,0.13333333333333333,0.13333333333333333

## TODO
- 週単位での販売履歴の平均に基づいた簡単な予測
- FacebookのProphetモデルの利用  
  https://facebook.github.io/prophet/
- 日の特徴量を元にしたLGBM/XGBモデル